In [106]:
import requests
import time
from pprint import pprint
import pandas as pd

In [119]:
def get_id_list(pageNumber):
    url = 'https://www.nstl.gov.cn/api/service/nstl/web/execute?target=nstl4.search4&function=paper/pc/list/pl'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    }
    data  = {
  'query': '{"c":10,"st":"0","f":[],"p":"","q":[{"k":"uni_s","a":1,"o":"","f":1,"v":"\u4E2D\u56FD\u79D1\u5B66\u9662\u5927\u5B66"},{"k":"yea","a":1,"o":"","f":1,"v":"2022"}],"op":"AND","s":["nstl","haveAbsAuK:desc","yea:desc","score"],"t":["DegreePaper"]}',
  'webDisplayId': '11',
  'sl': '1',
  'searchWordId': 'e380808e48fd873dd19c006d7a8501a3',
  'searchId': 'ca4b4bb73d24187981ce7e06b18a472d',
  'facetRelation': '[{"id":"a7e9bfe0358d69e152389c5c412eecc3","sequence":3,"field":"uni_s","name":"\u9662\u6821","value":"\u4E2D\u56FD\u79D1\u5B66\u9662\u5927\u5B66"},{"id":"c5612153a3c2a990483a48e5573f2c6e","sequence":2,"field":"yea","name":"\u5E74\u4EFD","value":"2022"}]',
  'pageSize': '10',
  'pageNumber': f'{pageNumber}'
}
    response = requests.post(url,headers=headers,data = data).json()
    data_list = response['data']
    id_list = [data[0]['v'] for data in data_list]
    return id_list

<Response [302]>


TypeError: 'Response' object is not subscriptable

In [108]:
def get_info(id):
    url = 'https://www.nstl.gov.cn/api/service/nstl/web/execute?target=nstl4.search4&function=paper/pc/detail'
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Origin': 'https://www.nstl.gov.cn',
        'Pragma': 'no-cache',
        'Referer': 'https://www.nstl.gov.cn/paper_detail.html?id=49eca30cf784e72e5dd259fc789bc898',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }
    data = {
        'id': f'{id}',
        'webDisplayId': '1001',
        'searchWordId': '5aaae7f21e66d420b6ca2aef49221ed4',
        'searchId': '6be74044bf3ec7d5fba04edb1639e3d8',
        'searchSequence': '1'
    }
    response = requests.post(url,headers=headers,data=data).json()
    data = response['data']
    f_to_v = {item['f']: item['v'] for item in data}
    
    return f_to_v

In [109]:
def get_info_list(id_list):
    info_list = []
    for id in id_list:
        f_to_v = get_info(id)
        info = {}
        info['title'] = f_to_v.get('tit', [''])[0]
        info['author'] = f_to_v.get('hasAut', [[{'v': ['']}]])[0][2].get('v', [''])[0]
        info['institution'] = f_to_v.get('hasCrOr', [[{'nam': ['']}]])[0][2].get('nam', [''])[0]
        info['university'] = f_to_v.get('uni', [''])[0]
        info['major'] = f_to_v.get('maj', [''])[0]
        info['degree'] = f_to_v.get('deg', [''])[0]
        info['department'] = f_to_v.get('off', [''])[0]
        info['tutor'] = f_to_v.get('hasTut', [[{'v': ['']}]])[0][2].get('v', [''])[0]
        info['language'] = f_to_v.get('lan', [{'zh': ''}])[0].get('zh', '')
        info['su_date'] = f_to_v.get('suda', [''])[0]
        info['qu_date'] = f_to_v.get('quda', [''])[0]
        info['classification_code'] = list(f_to_v.get('clco', [{}])[0].values())[0] if f_to_v.get('clco') else ''
        info['keywords'] = f_to_v.get('key', [])
        info['abstract'] = f_to_v.get('abs', [''])[0]

        info_list.append(info)
    return info_list

In [110]:
tit_list = []
hasAut_list = []
hasCrOr_list = []
uni_list = []
maj_list = []
deg_list = []
off_list = []
hasTut_list = []
lan_list = []
suda_list = []
quda_list = []
clco_list = []
key_list = []
abs_list = []

df = pd.DataFrame({
    '标题': tit_list,
    '作者': hasAut_list,
    '机构': hasCrOr_list,
    '院校': uni_list,
    '专业': maj_list,
    '学位': deg_list,
    '授予机构': off_list,
    '导师': hasTut_list,
    '语种': lan_list,
    '提交日期': suda_list,
    '论文答辩日期': quda_list,
    '分类号': clco_list,
    '关键词': key_list,
    '摘要': abs_list
})

In [111]:
id_list = get_id_list()
info_list = get_info_list(id_list)
for info in info_list:
    tit_list.append(info.get('title', ''))
    hasAut_list.append(info.get('author', ''))
    hasCrOr_list.append(info.get('institution', ''))
    uni_list.append(info.get('university', ''))
    maj_list.append(info.get('major', ''))
    deg_list.append(info.get('degree', ''))
    off_list.append(info.get('department', ''))
    hasTut_list.append(info.get('tutor', ''))
    lan_list.append(info.get('language', ''))
    suda_list.append(info.get('submission_date', ''))
    quda_list.append('')  # 如有对应数据请替换
    clco_list.append(info.get('classification_code', ''))
    key_list.append(', '.join(info.get('keywords', [])))
    abs_list.append(info.get('abstract', ''))
df.to_excel('论文信息汇总.xlsx', index=True)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)